线性回归区别

![简单线性回归和多元线性回归区别](./img/vs.png)

分类数据用dummy variable转换

![dummy variable](./img/dummy_var.png)

总是要省略一列防止过度拟合 (X-1)
![omit_one](./img/omit_one.png)


反向淘汰（Backward Elimination）是一种特征选择技术，常用于多元线性回归模型中。其基本思想是：先将所有的预测变量包含在模型中，然后逐步移除最不重要的特征，直到所有剩余的特征都是重要的（即对模型的预测性能有显著贡献）。

反向淘汰的具体步骤如下：

**包含所有特征：**首先，我们建立一个包含所有预测变量的模型。

**计算每个特征的重要性：**然后，我们计算每个特征对模型预测能力的贡献。在多元线性回归中，这通常通过计算每个特征的 P 值来实现。P 值表示了特征的系数是否显著不等于零的概率。较高的 P 值（通常大于某个阈值，如 0.05）表示特征可能不重要。

**移除最不重要的特征：**我们移除 P 值最大（即最不重要）的特征。

**重复步骤 2 和 3：**我们反复计算每个特征的重要性，然后移除最不重要的特征，直到所有剩余的特征都是重要的。

前向选择（Forward Selection）是一种特征选择技术，常用于多元线性回归模型中。其基本思想是：从没有预测变量的模型开始，然后逐步添加最重要的特征，直到添加更多的特征不能显著提高模型的预测性能。

前向选择的具体步骤如下：

**开始没有特征的模型：**首先，我们建立一个没有预测变量的模型，只有一个截距。

**计算每个特征的重要性：**然后，我们计算每个特征对模型预测能力的贡献。在多元线性回归中，这通常通过计算每个特征的 P 值来实现。P 值表示了特征的系数是否显著不等于零的概率。较低的 P 值（通常小于某个阈值，如 0.05）表示特征可能是重要的。

**添加最重要的特征：**我们添加 P 值最小（即最重要）的特征。

**重复步骤 2 和 3：**我们反复计算每个特征的重要性，然后添加最重要的特征，直到添加更多的特征不能显著提高模型的预测性能。

双向淘汰（Bidirectional Elimination），也被称为逐步回归（Stepwise Regression），是一种特征选择方法，结合了前向选择和反向淘汰的策略。它在每一步都会评估添加新特征或者删除已有特征对模型性能的影响，然后选择最好的策略。

双向淘汰的具体步骤如下：

**开始没有特征的模型：**首先，我们建立一个没有预测变量的模型，只有一个截距。

**前向选择：**然后，我们像前向选择一样，逐步添加最重要的特征，直到添加更多的特征不能显著提高模型的预测性能。

**反向淘汰：**在每次添加新特征后，我们像反向淘汰一样，检查是否有已有的特征变得不重要（例如，它的 P 值变得较高）。如果有，我们就移除这个特征。

**重复步骤 2 和 3：**我们反复进行前向选择和反向淘汰，直到不能添加或移除任何特征。

In [36]:
import numpy as np
import matplotlib as plt
import pandas as pd

In [37]:
dataset = pd.read_csv('50_Startups.csv')
dataset.head(10)

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94
5,131876.90,99814.71,362861.36,New York,156991.12
6,134615.46,147198.87,127716.82,California,156122.51
7,130298.13,145530.06,323876.68,Florida,155752.60
8,120542.52,148718.95,311613.29,New York,152211.77
9,123334.88,108679.17,304981.62,California,149759.96


## 选取除了最后一列的所有列，并将选取的数据转换为一个 NumPy 数组作为自变量X

In [38]:
X = dataset.iloc[:, :-1].values

## 选取最后一列，并将选取的数据转换为一个 NumPy 数组作为因变量y

In [39]:
y = dataset.iloc[:, 4].values

处理分类数据

In [40]:
from sklearn.preprocessing import OneHotEncoder
onehotencoder = OneHotEncoder()
X_reshaped = X[:, 3].reshape(-1, 1)  # 第三列地点，转换为二维数组
X_onehot = onehotencoder.fit_transform(X_reshaped).toarray()  # 对第一列进行 one-hot 编码

处理虚拟变量陷阱，去除第一列

In [41]:
X_onehot = X_onehot[:, 1:]

去除掉最后一列(地址)， 将编码的结果合并在一起：

In [42]:
X_new = np.concatenate([X_onehot, X[:, :-1]], axis=1)
X_new

array([[0.0, 1.0, 165349.2, 136897.8, 471784.1],
       [0.0, 0.0, 162597.7, 151377.59, 443898.53],
       [1.0, 0.0, 153441.51, 101145.55, 407934.54],
       [0.0, 1.0, 144372.41, 118671.85, 383199.62],
       [1.0, 0.0, 142107.34, 91391.77, 366168.42],
       [0.0, 1.0, 131876.9, 99814.71, 362861.36],
       [0.0, 0.0, 134615.46, 147198.87, 127716.82],
       [1.0, 0.0, 130298.13, 145530.06, 323876.68],
       [0.0, 1.0, 120542.52, 148718.95, 311613.29],
       [0.0, 0.0, 123334.88, 108679.17, 304981.62],
       [1.0, 0.0, 101913.08, 110594.11, 229160.95],
       [0.0, 0.0, 100671.96, 91790.61, 249744.55],
       [1.0, 0.0, 93863.75, 127320.38, 249839.44],
       [0.0, 0.0, 91992.39, 135495.07, 252664.93],
       [1.0, 0.0, 119943.24, 156547.42, 256512.92],
       [0.0, 1.0, 114523.61, 122616.84, 261776.23],
       [0.0, 0.0, 78013.11, 121597.55, 264346.06],
       [0.0, 1.0, 94657.16, 145077.58, 282574.31],
       [1.0, 0.0, 91749.16, 114175.79, 294919.57],
       [0.0, 1.0, 86419.7

拆分训练和测试数据

In [43]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=0)

拟合模型

In [44]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()

regressor.fit(X_train, y_train)

LinearRegression()

测试预测结果

In [45]:
y_pred = regressor.predict(X_test)